# Project Title
### Data Engineering Capstone Project

#### Project Summary
To Describe the average climate in Different states or on Different year

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
import pandas as pd
import os
import glob
import datetime
from pyspark.sql import SparkSession, SQLContext, GroupedData
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [46]:
output_data='/home/workspace/output/'

In [3]:
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()

### Step 1: Scope the Project and Gather Data

#### Scope 
Plan is to explore the file GlobalLandTemperaturesByCity and the airport-codes_csv.csv and the us-cities-demographics.csv to get the relevent info regarding the cities and states in United States

#### Describe and Gather Data 
1. GlobalLandTemperaturesByCity - This file inclueds temperature info by Cities in different point in time
2. airport-codes_csv.csv - This file contains the list of airport codes based on the Cities and the kinds of airports
3. us-cities-demographics.csv - This file has demographic informartion for US-Cities

In [4]:
demog=spark.read.option("delimiter", ";").csv("us-cities-demographics.csv",header=True)
airport=spark.read.csv("airport-codes_csv.csv",header=True)
temperatureData=spark.read.csv("../../data2/GlobalLandTemperaturesByCity.csv",header=True)

### Step 2: Explore and Assess the Data
#### Explore the Data 
The data has null values for specific Cities

#### Cleaning Steps
Dropped the Null values


In [5]:
temperatureData.printSchema()

root
 |-- dt: string (nullable = true)
 |-- AverageTemperature: string (nullable = true)
 |-- AverageTemperatureUncertainty: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)



In [39]:
#filter the world Temperature Data for only the U.S. and drop duplicates and convert celcius temperature to fahrenheit
usTemperatures=temperatureData.filter(temperatureData["Country"]=="United States")\
.withColumn("year",F.year(temperatureData["dt"]))\
.withColumn("month",F.month(temperatureData["dt"]))\
.withColumn('start_time',F.to_date(temperatureData["dt"]))\
.withColumn("avg_temp_fahrenheit",temperatureData["AverageTemperature"]*9/5+32)\

new_Temperatures=usTemperatures.select("year","month",F.round(F.col("AverageTemperature"),1).alias("avg_temp_celcius"),\
                    F.round(F.col("avg_temp_fahrenheit"),1).alias("avg_temp_fahrenheit"),"City","Country","start_time") \
                    .dropDuplicates()

In [7]:
#new_Temperatures.groupBy([F.year(new_Temperatures["start_time"])]).count().orderBy(F.desc('count')).take(5)

In [8]:
#new_Temperatures.groupBy(['City']).count().orderBy(F.desc('count')).take(5)

[Row(City='Springfield', count=9364),
 Row(City='Columbus', count=6358),
 Row(City='Aurora', count=6079),
 Row(City='Arlington', count=5563),
 Row(City='Peoria', count=5384)]

In [9]:
#new_Temperatures.filter(new_Temperatures.City=='Newark').take(5)

[Row(year=1756, month=5, avg_temp_celcius=None, avg_temp_fahrenheit=None, City='Newark', Country='United States', dt='1756-05-01'),
 Row(year=1767, month=7, avg_temp_celcius=22.1, avg_temp_fahrenheit=71.8, City='Newark', Country='United States', dt='1767-07-01'),
 Row(year=1771, month=2, avg_temp_celcius=-4.0, avg_temp_fahrenheit=24.8, City='Newark', Country='United States', dt='1771-02-01'),
 Row(year=1772, month=1, avg_temp_celcius=-3.7, avg_temp_fahrenheit=25.3, City='Newark', Country='United States', dt='1772-01-01'),
 Row(year=1774, month=6, avg_temp_celcius=20.0, avg_temp_fahrenheit=68.0, City='Newark', Country='United States', dt='1774-06-01')]

In [41]:
new_Temperatures = new_Temperatures \
.join(demog.select('City','State').distinct(),on='City',how='inner')

### Climate info by City

In [42]:
new_Temperatures = new_Temperatures.filter(new_Temperatures.avg_temp_celcius.isNotNull())\
                        .filter(new_Temperatures.avg_temp_fahrenheit.isNotNull())

In [43]:
new_Temperatures.take(5)

[Row(City='Abilene', year=1836, month=9, avg_temp_celcius=21.9, avg_temp_fahrenheit=71.4, Country='United States', start_time=datetime.date(1836, 9, 1), State='Texas'),
 Row(City='Abilene', year=1837, month=2, avg_temp_celcius=7.7, avg_temp_fahrenheit=45.9, Country='United States', start_time=datetime.date(1837, 2, 1), State='Texas'),
 Row(City='Abilene', year=1849, month=7, avg_temp_celcius=27.5, avg_temp_fahrenheit=81.5, Country='United States', start_time=datetime.date(1849, 7, 1), State='Texas'),
 Row(City='Abilene', year=1850, month=8, avg_temp_celcius=28.8, avg_temp_fahrenheit=83.9, Country='United States', start_time=datetime.date(1850, 8, 1), State='Texas'),
 Row(City='Abilene', year=1866, month=11, avg_temp_celcius=11.1, avg_temp_fahrenheit=51.9, Country='United States', start_time=datetime.date(1866, 11, 1), State='Texas')]

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
1. dim_time
        - Year
        - Month
        - Date
2. dim_city
        - City
        - State
        - Country
3. fact_temp
        - City
        - Date
        - avg_temp_fahrenheit
        - avg_temp_celcius

#### 3.2 Mapping Out Data Pipelines
Joining the new_Temperatures dataframe with demographics dataframe on City

1. The fact table is loaded with partition by City, start_time on date to local storage.
2. City Table is loaded with partitons by Country and state
3. Time table is loaded with partitions by year
4. Extract transformed state demographical information from city demographics table as demographics dimension table.
5. Converted the dt from to pyspark Date type format
6. Extracted the year and month from the Temperatures file
7. Created time table based on year, month, start time 
8. Created City table based on the City and Countries mentioned in the Temperatures file and demographics file for United States

In [ ]:
new_Temperatures = new_Temperatures \
.join(demog.select('City','State').distinct(),on='City',how='inner')

## Time table write to parquet

In [50]:
time_table = new_Temperatures.select(['year','month','start_time']).distinct()
time_table.write.parquet(output_data+"time/",  mode="overwrite",partitionBy=["year"])

## City Table write to parquet

In [51]:
city_meta = new_Temperatures.select(['Country','State','City']).distinct()
city_meta.write.parquet(output_data+"city_meta/",  mode="overwrite",partitionBy=["Country","State"])

## Fact Temperature Table write to parquet

In [ ]:
fact_temp = new_Temperatures.select(['avg_temp_celcius','avg_temp_fahrenheit','City','start_time'])
fact_temp.write.parquet(output_data+"fact_temp/",  mode="overwrite",partitionBy=["City","start_time"])

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.
Assuming if a new dataset for a new year comes in 
Steps 
1. Derive the time_table columns
2. Join the dataframe to demographics dataset in City
3. Overwrite the existing table for new set of data
4. Using the Star Schema 
    1. The temperature information is based on time and City that is added in the fact table
    2. The City is unique list of City for a given country from Temperatures information as we are only looking for united states it is highly  related to the Cities mentioned in the demographics file to get the State name
    3. Time table is derived from the dt in the Temperatures file which has the start time , month, year
    4. As the users can have flexibility to look the temperature based on States as well as Year or month

In [ ]:
temperatureData2=spark.read.csv("../../data2/GlobalLandTemperaturesByCity.csv",header=True)
demog2=spark.read.option("delimiter", ";").csv("us-cities-demographics.csv",header=True)

In [39]:
#filter the world Temperature Data for only the U.S. and only == 2013 and drop duplicates and convert celcius temp to f
usTemperatures2=temperatureData2.filter(temperatureData2["Country"]=="United States")\
.withColumn("year",F.year(temperatureData2["dt"]))\
.withColumn("month",F.month(temperatureData2["dt"]))\
.withColumn('start_time',F.to_date(temperatureData2["dt"]))\
.withColumn("avg_temp_fahrenheit",temperatureData2["AverageTemperature"]*9/5+32)\

new_Temperatures2=usTemperatures2.select("year","month",F.round(F.col("AverageTemperature"),1).alias("avg_temp_celcius"),\
                    F.round(F.col("avg_temp_fahrenheit"),1).alias("avg_temp_fahrenheit"),"City","Country","start_time") \
                    .dropDuplicates()

In [41]:
new_Temperatures2 = new_Temperatures2 \
.join(demog2.select('City','State').distinct(),on='City',how='inner')

In [ ]:
fact_temp2 = new_Temperatures2.select(['avg_temp_celcius','avg_temp_fahrenheit','City','start_time'])
fact_temp2.write.parquet(output_data+"fact_temp/",  mode="overwrite",partitionBy=["City","start_time"])

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [53]:
city_meta.count()

261

In [54]:
city_meta.dropDuplicates().count()

261

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

In [2]:
!cat Data-Dictionary.md

DataDictionary

Time table
1. start_time - DateType notnull primary key - date on which the temperature was measured
2. month - integer  notnull - month on which the temperature was measured
3. year - integer  notnull - year on which the temperature was measured

City Table
1. City - string notnull primary key - avg temperature of the City
2. State - string notnull - State in which the City is locate
3. Country - string not null - Country in which the State is located

fact_temp
1. start_time - string notnull primary_key - date on which the temperature was measured
2. City - string notnull - City in which the temperature was measured 
3. avg_temp_fahrenheit - integer notnull - avg temperature in fahrenheit
4. avg_temp_celcius - integer notnull - avg temperature in celcius

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.

1. Spark is primarily used for data exploration and ETL for faster processing time

2. The data can be updated as and when the temprature for new year is available to load

3. Currently this ETL process is only using 1 node to process the Temperature data. If the data grows I would mostly us a distributed architecture by scaling out (increasing the number of nodes) and also if that is not optimal then I would also increase the type of nodes (increasing the memory/cpu for each node in cluster) to reduce the processing time

4. I would create a spark job and run it when new data is available. I would check the data source daily4. 

5. I would check the types of queries that 100+ people would like to run based on which I would use Redshift (for ad hoc querying) or Apache Cassandra(for daily reports)

#### Purpose of the Final Data Model

What will the data be used for? Who will use it?

1. Data Model would be used by different users for Daily weather check based on the user City also Could be used for Temperature comparisons between different cities and comparisons of the cities can be at the State level or Country level also base or time (year, month, day)